In [1]:
import time
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [2]:
# Preparing for Data
# print('==> Preparing data..')

# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


# classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [3]:
def tb_writer(optim):
    #timestamp = time.strftime('%Y%m%d_%H_%M_%S')
    timestamp = time.strftime(optim)
    writer = SummaryWriter(log_dir="F:\PythonProject\ECE579_Intro_to_DL\HW4\log_dir/"+timestamp)
    return writer

In [4]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        ############################
        #### Put your code here ####
        ############################
        self.model = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 120, kernel_size=5),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(in_features=120, out_features=84),
            nn.ReLU(),
            nn.Linear(in_features=84, out_features=10)
        )
        ###########################
        #### End of your codes ####
        ###########################

    def forward(self, x):
        ############################
        #### Put your code here ####
        ############################
        x = self.model(x)
        ###########################
        #### End of your codes ####
        ###########################
        return x

In [5]:
class LeNetWithDropout(nn.Module):
    def __init__(self):
        super(LeNetWithDropout, self).__init__()
        ############################
        #### Put your code here ####
        ############################
        self.model = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 120, kernel_size=5),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(in_features=120, out_features=84),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(in_features=84, out_features=10)
        )
        ###########################
        #### End of your codes ####
        ###########################

    def forward(self, x):
        ############################
        #### Put your code here ####
        ############################
        x = self.model(x)
        ###########################
        #### End of your codes ####
        ###########################
        return x

In [6]:
class LeNetWithBN(nn.Module):
    def __init__(self):
        super(LeNetWithBN, self).__init__()
        ############################
        #### Put your code here ####
        ############################
        self.model = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 120, kernel_size=5),
            nn.BatchNorm2d(120),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(in_features=120, out_features=84),
            nn.ReLU(),
            nn.Linear(in_features=84, out_features=10)
        )
        ###########################
        #### End of your codes ####
        ###########################

    def forward(self, x):
        ############################
        #### Put your code here ####
        ############################
        x = self.model(x)
        ###########################
        #### End of your codes ####
        ###########################
        return x

In [7]:
def train(model, device, train_loader, optimizer, epoch, criterion, writer):
    model.train()
    total_loss = 0.
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        ############################
        #### Put your code here ####
        ############################
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss
    train_loss = total_loss/len(train_loader.dataset)
    print("Epoch :{}\tLoss :{:.6f}".format(epoch, train_loss))
    writer.add_scalar("Train Loss", train_loss, epoch)
    writer.flush()
    # return train_loss
        ###########################
        #### End of your codes ####
        ###########################
        # if batch_idx % 10 == 0:
        #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #         epoch, batch_idx * len(data), len(train_loader.dataset),
        #                100. * batch_idx / len(train_loader), loss.item()/len(train_loader.dataset)))

In [8]:
def test(model, device, test_loader, criterion, epoch, writer):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            total += target.size(0)
        correct /= total
        test_loss /= len(test_loader.dataset)
        print("Average Test Loss :{:.4f}, Accuracy :{:.2f}%\n".format(test_loss, correct * 100.0))
        writer.add_scalar("Test Loss", test_loss, epoch)
        writer.add_scalar("Accuracy", correct, epoch)
        writer.flush()
        # return test_loss, correct
    # print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    #     test_loss, correct, len(test_loader.dataset),
    #     100. * correct / len(test_loader.dataset)))

In [9]:
time0 = time.time()
# Training settings
batch_size = 64
epochs = 50
lr = 0.001
save_model = False
torch.manual_seed(100)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

models = {"LeNet": LeNet(), "LeNet With Dropout":LeNetWithDropout(), "LeNet With BN": LeNetWithBN()}
criterion = nn.CrossEntropyLoss()

for model_name in models:
    log_writer = tb_writer(model_name)
    model = models[model_name].to(device=device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch, criterion, log_writer)
        test(model, device, test_loader, criterion, epoch, log_writer)
        log_writer.close()
# if (save_model):
#     torch.save(model.state_dict(), "cifar_lenet.pt")
time1 = time.time()
print('Traning and Testing total excution time is: %s seconds ' % (time1 - time0))

Files already downloaded and verified
Files already downloaded and verified
Epoch :1	Loss :0.017831
Average Test Loss :0.0222, Accuracy :20.25%

Epoch :2	Loss :0.016403
Average Test Loss :0.0197, Accuracy :29.48%

Epoch :3	Loss :0.015222
Average Test Loss :0.0185, Accuracy :33.29%

Epoch :4	Loss :0.014381
Average Test Loss :0.0173, Accuracy :36.84%

Epoch :5	Loss :0.013771
Average Test Loss :0.0165, Accuracy :39.60%

Epoch :6	Loss :0.013330
Average Test Loss :0.0160, Accuracy :41.70%

Epoch :7	Loss :0.012875
Average Test Loss :0.0156, Accuracy :42.88%

Epoch :8	Loss :0.012541
Average Test Loss :0.0150, Accuracy :45.07%

Epoch :9	Loss :0.012231
Average Test Loss :0.0149, Accuracy :46.13%

Epoch :10	Loss :0.012014
Average Test Loss :0.0144, Accuracy :47.54%

Epoch :11	Loss :0.011816
Average Test Loss :0.0142, Accuracy :48.24%

Epoch :12	Loss :0.011646
Average Test Loss :0.0139, Accuracy :49.35%

Epoch :13	Loss :0.011427
Average Test Loss :0.0138, Accuracy :50.41%

Epoch :14	Loss :0.01123